In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [2]:
load_dotenv(override=True)

True

In [3]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [6]:
documents = recur_split.split_documents(file)

# Embbiding (OpenAI)

In [7]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [8]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# LLM (OpenAI)

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [10]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [11]:
question = "O que é Identificar oportunidades?"

# Answer

In [12]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?',
 'result': 'Identificar oportunidades é usar a imaginação e as habilidades para visualizar oportunidades de criação de valor, como a identificação de necessidades e desafios que precisam ser atendidos.'}

# Prompt

In [13]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
- A disciplina;
- O conteúdo;
- O objetivo da aula.

Com base no documento ENTRECOMP (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Retorne apenas os **nomes das competências**, separados por vírgula e a última separada por e.
Exemplo de formato: Visão, Pensamento ético e sustentável e Valorizar ideias.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

# Chain

In [14]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [15]:
question = "Aula de <Educação Religiosa>, sendo o conteúdo <Nomes> e o objetivo da aula é <Importância dos Nomes>."

# Answer

In [16]:
answer = chat_chain.invoke({"query": question})

In [17]:
print(answer["result"])

Visão, Valorizar ideias e Consciência de si e autoeficácia.
